In [33]:
# data processing
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# regression / matching
import statsmodels.formula.api as smf

## Load the data : EDAC and magneto

In [34]:
data_path = "DATA/"

vex_df = pd.read_csv(data_path+"VENUS_EXPRESS/VEX_NDMW0D0A_2023_01_11_12_55_55.900.txt", sep='\t')
mag_df = pd.read_csv(data_path+"VEX_MAGNETO/VEX-V-Y-MAG-4.csv")

We calculate the counts per day and remove the resetting of the counter at 0 for EDAC:

In [35]:
#vex_df['DATE_TIME'] = vex_df.DATE_TIME.apply(lambda d: datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f')) #takes more time for the graphs to show
vex_df['EDAC'] = vex_df['NDMW0D0A'].diff().bfill().astype(int).map(lambda x: max(0, x))
vex_df.drop('NDMW0D0A', axis=1, inplace=True)

In [36]:
vex_df.head()

,DATE_TIME,EDAC
0,2005-11-09 00:09:04.575,0
1,2005-11-09 00:10:08.575,0
2,2005-11-09 00:11:12.576,0
3,2005-11-09 00:12:16.576,0
4,2005-11-09 00:13:20.577,0


In [37]:
mag_df.head()

,date,BX,BY,BZ,BT,XSC,YSC,ZSC,RSC
0,2006-04-24 00:00:00,12.357658,-1.304164,-3.332425,12.881274,-4451.055178,-3196.485753,-65466.762260,65695.760575
1,2006-04-24 00:05:00,12.868947,-0.980800,-3.360027,13.340680,-4202.246280,-3138.377907,-65806.350827,66015.078600
2,2006-04-24 00:10:00,12.857438,-0.871986,-3.487877,13.355384,-3954.000329,-3080.233288,-66137.913808,66327.612616
3,2006-04-24 00:15:00,12.898635,-0.684986,-2.885689,13.248405,-3705.057257,-3021.761270,-66463.291041,66635.079608
4,2006-04-24 00:20:00,12.766473,-0.517608,-2.217135,12.972905,-3453.676541,-2962.553108,-66784.717784,66939.596338


## Grouping by 5 minutes and merge

In [38]:
# Convert the 'DATE_TIME' column in vex_df to datetime and set as index
vex_df['DATE_TIME'] = pd.to_datetime(vex_df['DATE_TIME'])
vex_df.set_index('DATE_TIME', inplace=True)

# Resample and take the mean for each 5-minute period
vex_df_resampled = vex_df.resample('5T').sum()

# Reset the index so 'DATE_TIME' becomes a column again
vex_df_resampled.reset_index(inplace=True)

# For mag_df, ensure the 'date' column is in datetime format and set as index
mag_df['date'] = pd.to_datetime(mag_df['date'])
mag_df.set_index('date', inplace=True)

In [39]:
vex_df_resampled.head()

,DATE_TIME,EDAC
0,2005-11-09 00:05:00,0
1,2005-11-09 00:10:00,0
2,2005-11-09 00:15:00,0
3,2005-11-09 00:20:00,0
4,2005-11-09 00:25:00,0


In [40]:
# Now, join the two dataframes on their datetime indices
df = pd.merge_asof(vex_df_resampled, mag_df, left_on='DATE_TIME', right_index=True)

In [41]:
# Drop the rows with NaNs
df.dropna(inplace=True)

In [42]:
df.head()

,DATE_TIME,EDAC,BX,BY,BZ,BT,XSC,YSC,ZSC,RSC
47807,2006-04-24 00:00:00,0,12.357658,-1.304164,-3.332425,12.881274,-4451.055178,-3196.485753,-65466.762260,65695.760575
47808,2006-04-24 00:05:00,0,12.868947,-0.980800,-3.360027,13.340680,-4202.246280,-3138.377907,-65806.350827,66015.078600
47809,2006-04-24 00:10:00,0,12.857438,-0.871986,-3.487877,13.355384,-3954.000329,-3080.233288,-66137.913808,66327.612616
47810,2006-04-24 00:15:00,0,12.898635,-0.684986,-2.885689,13.248405,-3705.057257,-3021.761270,-66463.291041,66635.079608
47811,2006-04-24 00:20:00,0,12.766473,-0.517608,-2.217135,12.972905,-3453.676541,-2962.553108,-66784.717784,66939.596338


## Old : grouping by hour
Since the magneto data is grouped by hour, we aggregate the EDAC data by hour too:

In [43]:
# first we group byhour:
# vex_df['DATE_TIME'] = pd.to_datetime(vex_df['DATE_TIME'])
# hourly_grouped = vex_df.groupby(pd.Grouper(key='DATE_TIME', freq='H'))

In [44]:
# sanity check:
# first_group = hourly_grouped.get_group(list(hourly_grouped.groups.keys())[0])
# print(first_group)

In [45]:
# then we agregate the groups:
# df = hourly_grouped.agg({'DATE_TIME': lambda x: x.dt.round('H').min(),'EDAC': 'sum'})

In [46]:
# we drop lines containing NaNs, then check that no Nans are left in our dataframe:
# df.dropna(inplace=True)
# 
# has_nan = df.isna().any().any()
# print(has_nan)

In [47]:
# sanity check:
# df.head()

## Adding CME labels
Now let's add the events of CME :

In [48]:
vex_cme_df = pd.read_csv(data_path+"VENUS_EXPRESS/VEX_CME_date_time.csv")
df['cme'] = 0

In [49]:
vex_cme_df.start_time = pd.to_datetime(vex_cme_df.start_time)
vex_cme_df.end_time = pd.to_datetime(vex_cme_df.end_time)

In [50]:
for index, row in vex_cme_df.iterrows():
    start_time = row['start_time'].round('5T')
    end_time = row['end_time'].round('5T')
    
    # Set 'cme' column values to 1 between start_time and end_time
    df.loc[(df.DATE_TIME >= start_time) & (df.DATE_TIME <= end_time), 'cme'] = 1

In [51]:
# sanity check:
df.head()

,DATE_TIME,EDAC,BX,BY,BZ,BT,XSC,YSC,ZSC,RSC,cme
47807,2006-04-24 00:00:00,0,12.357658,-1.304164,-3.332425,12.881274,-4451.055178,-3196.485753,-65466.762260,65695.760575,0
47808,2006-04-24 00:05:00,0,12.868947,-0.980800,-3.360027,13.340680,-4202.246280,-3138.377907,-65806.350827,66015.078600,0
47809,2006-04-24 00:10:00,0,12.857438,-0.871986,-3.487877,13.355384,-3954.000329,-3080.233288,-66137.913808,66327.612616,0
47810,2006-04-24 00:15:00,0,12.898635,-0.684986,-2.885689,13.248405,-3705.057257,-3021.761270,-66463.291041,66635.079608,0
47811,2006-04-24 00:20:00,0,12.766473,-0.517608,-2.217135,12.972905,-3453.676541,-2962.553108,-66784.717784,66939.596338,0


In [52]:
cme_count = df[df['cme'] == 1].shape[0]
print(f"Number of rows where cme is equal to 1: {cme_count}")

Number of rows where cme is equal to 1: 15939


##### Parenthesis : class imbalance
Let's have a look at the proportion of CME events: we see that only 1.8% of our data corresponds to a CME.

Since we have this huge class imbalance, we can expect the models we will train to be biased towards predicting no CMEs.
We will have to implement various methods to mitigate this class imbalance.

In [53]:
cme_count = df[df['cme'] == 1].shape[0]
total_count = len(df)
percentage = (cme_count / total_count) * 100

print(f"CME count: {cme_count}")
print(f"Total count: {total_count}")
print(f"Percentage of CME events: {percentage:.2f}%")


CME count: 15939
Total count: 893069
Percentage of CME events: 1.78%


# Check the data seb

In [54]:
df.head()
df2 = df.copy()
df2.set_index('DATE_TIME', inplace=True)
df2 = df2.resample('5min').mean() 
df2.reset_index(inplace=True)

0.0 Percentage of nan values
1.2336544926920763 Percentage of nan values
893069 904224


End of parenthesis, back to our dataframe:

We save this data in a first csv that we will use to train our models (predict CME events based on EDAC):

In [57]:
train_file_path = "DATA/training_data/"
df2.to_csv(train_file_path+'VEX_edac_mag_labeled_nan.csv', index=False)

## Old : Adding magneto data
Now let's add the magneto data to a second dataframe (to assess the performance of our predictor: we will see how much gap in performance we have if we predict CME events based on EDAC and magneto data):

In [58]:
# Load magneto data
magneto_path = "DATA/VEX_MAGNETO/VEX-V-Y-MAG-4.csv"
magneto_df = pd.read_csv(magneto_path)

In [59]:
# we also set the date as index to merge it with our EDAC dataframe
magneto_df.rename(columns={'date': 'DATE_TIME'}, inplace=True)
magneto_df.set_index('DATE_TIME', inplace=True)

# we convert the index to datetime to be able to merge it with our EDAC dataframe
magneto_df.index = pd.to_datetime(magneto_df.index)

In [60]:
# sanity check:

print("magneto_df index type:", magneto_df.index.dtype)
print("df index type:", df.index.dtype)

magneto_df index type: datetime64[ns]
df index type: int64


In [61]:
magneto_df.head()

,BX,BY,BZ,BT,XSC,YSC,ZSC,RSC
DATE_TIME,,,,,,,,
2006-04-24 00:00:00,12.357658,-1.304164,-3.332425,12.881274,-4451.055178,-3196.485753,-65466.762260,65695.760575
2006-04-24 00:05:00,12.868947,-0.980800,-3.360027,13.340680,-4202.246280,-3138.377907,-65806.350827,66015.078600
2006-04-24 00:10:00,12.857438,-0.871986,-3.487877,13.355384,-3954.000329,-3080.233288,-66137.913808,66327.612616
2006-04-24 00:15:00,12.898635,-0.684986,-2.885689,13.248405,-3705.057257,-3021.761270,-66463.291041,66635.079608
2006-04-24 00:20:00,12.766473,-0.517608,-2.217135,12.972905,-3453.676541,-2962.553108,-66784.717784,66939.596338


We merge this data with our first dataframe of ENACs and CMEs.

We will have many NaNs because the magneto data covers a shorter time span than the EDAC data : we will remove those.

In [62]:
merged_df = df.merge(magneto_df, left_index=True, right_index=True, how='left')

In [63]:
merged_df.dropna(inplace=True) # drop lines containing NaNs
merged_df.head()

,DATE_TIME,EDAC,BX_x,BY_x,BZ_x,BT_x,XSC_x,YSC_x,ZSC_x,RSC_x,cme,BX_y,BY_y,BZ_y,BT_y,XSC_y,YSC_y,ZSC_y,RSC_y


To train many models, it is easier to deal with numerical values. We will transform `DATE_TIME` into `delta_hour`.

In [64]:
merged_df['delta_hour'] = merged_df['DATE_TIME'].diff().dt.total_seconds() / 3600.0
merged_df.dropna(inplace=True) # the first line will have a NaN delta_hour, so we drop it

In [65]:
merged_df.drop('DATE_TIME', axis=1, inplace=True)
merged_df.head()

,EDAC,BX_x,BY_x,BZ_x,BT_x,XSC_x,YSC_x,ZSC_x,RSC_x,cme,BX_y,BY_y,BZ_y,BT_y,XSC_y,YSC_y,ZSC_y,RSC_y,delta_hour


We save this data in a second csv that we will use to train the same models.

This will allow to assess the performance of our predictor: we will see how much gap in performance we have if we predict CME events based on only EDAC data compared to predicting it using both EDAC and magneto data

In [66]:
# merged_df.to_csv(train_file_path+'VEX_edac_mag_labeled.csv', index=False)

In [67]:
merged_df.head()

,EDAC,BX_x,BY_x,BZ_x,BT_x,XSC_x,YSC_x,ZSC_x,RSC_x,cme,BX_y,BY_y,BZ_y,BT_y,XSC_y,YSC_y,ZSC_y,RSC_y,delta_hour
